# Lab 9 Group 21
## Group Members: (Andrew Quijano, afq2003), (Isaiah Genis | ig596), (Jamie Leach | jhl925), (Yevhen Pankevych | yp2525)


## Vuln 1: Improper Password validation and secure-by-default handling.
SampleNetworkServer.py does not properly handle commands chained to invalid AUTH and Logout requests. An attacker can submit a invalid password to the AUTH command and still be allowed to execute all other commands such as SET_DEGF or UPDATE_TEMP that would impact the legitimate client. This is a direct impact on the confidentiality of the system as it can be access by anyone.

This occurs on lines 59-72 in the processCommands function.

The Patch for this vulnerability is to add proper error handling of invalid auth tokens to the processCommands function of SampleNetworkServer.py. See lines 13-15 of the below method
### Exploit of Bad Password Error Handling

In [92]:
# Exploit
import unittest
import socket

def is_number(s):
    try:
        float(s)
        return True
    except ValueError:
        return False
def run_specified_tests(tests:list):
    # Run only the tests in the specified classes
    test_classes_to_run = tests
    loader = unittest.TestLoader()
    suites_list = []
    for test_class in test_classes_to_run:
        suite = loader.loadTestsFromTestCase(test_class)
        suites_list.append(suite)
    big_suite = unittest.TestSuite(suites_list)
    runner = unittest.TextTestRunner()
    results = runner.run(big_suite)
    return results

class TestPassword(unittest.TestCase):
    # Vuln 1 Exploit Test
    def test_bad_password(self):
        s_rl = socket.socket(family=socket.AF_INET, type=socket.SOCK_DGRAM)
        input_request=b'AUTH basd;GET_TEMP'
        s_rl.sendto(input_request, ("127.0.0.1", 23456))
        output_responce, real_server_addr = s_rl.recvfrom(1024)
        output_responce=str(output_responce.strip(),"utf8")
        s_rl.close()
        # print(output_responce)
        self.assertFalse(is_number(output_responce),msg="Received Valid Temp when should have been rejected for invalid auth.")
if __name__ == '__main__':
    run_specified_tests([TestPassword])

x
----------------------------------------------------------------------
Ran 1 test in 0.052s

OK (expected failures=1)


### Patched AUTH Error Handling.

In [85]:
# Patched Function Snippet
import string
import random

def processCommands(self, msg, addr) :
    cmds = msg.split(';')
    for c in cmds :
        cs = c.split(' ')
        if len(cs) == 2 : #should be either AUTH or LOGOUT
            if cs[0] == "AUTH":
                if cs[1] == "!Q#E%T&U8i6y4r2w" :
                    self.tokens.append(''.join(random.choice(string.ascii_uppercase + string.ascii_lowercase + string.digits) for _ in range(16)))
                    self.serverSocket.sendto(self.tokens[-1].encode("utf-8"), addr)
                else:
                    self.serverSocket.sendto(b"Bad Token\n", addr)
                    return
                    #print (self.tokens[-1])
            elif cs[0] == "LOGOUT":
                if cs[1] in self.tokens :
                    self.tokens.remove(cs[1])
            else : #unknown command
                self.serverSocket.sendto(b"Invalid Command\n", addr)
        elif c == "SET_DEGF" :
            self.deg = "F"
        elif c == "SET_DEGC" :
            self.deg = "C"
        elif c == "SET_DEGK" :
            self.deg = "K"
        elif c == "GET_TEMP" :
            self.serverSocket.sendto(b"%f\n" % self.getTemperature(), addr)
        elif c == "UPDATE_TEMP" :
            self.updateTemperature()
        elif c :
            self.serverSocket.sendto(b"Invalid Command\n", addr)

## Vuln 2: Ability to chain commands on Logout after token is revoked.

Calling the Logout command should immediately suspend my access to configurations and resources. Failing to exit allows the user to maintain access to sensitive functions and information.

This occurs on lines 59-72 in the processCommands function.

The Patch for this vulnerability is to check that before executing any of the commands that there is a successful AUTH completed first.

- Related Controls: NIST 800-53v5 AC-12(Session Termination)
### Exploit of Logout chaining

In [86]:
#Exploit
import unittest
import socket


class TestLogout(unittest.TestCase):
     # Logout Test
    def test_bad_logout(self):
        s_rl = socket.socket(family=socket.AF_INET, type=socket.SOCK_DGRAM)
        request_1 = b'AUTH !Q#E%T&U8i6y4r2w;'
        s_rl.sendto(request_1, ("127.0.0.1", 23456))
        token, real_server_addr = s_rl.recvfrom(1024)
        token = token.strip()
        # print(token[-1])
        request_2 = bytes(f"LOGOUT {token};GET_TEMP","utf8")
        s_rl.sendto(request_2, ("127.0.0.1", 23456))
        output_responce, real_server_addr = s_rl.recvfrom(1024)
        output_responce = str(output_responce.strip(),"utf8")
        s_rl.close()
        # print(output_responce)
        self.assertFalse(is_number(output_responce),msg="Received Valid Temp when should have been rejected for invalid token.")
# res = unittest.main(argv=[''], verbosity=3, exit=False)
if __name__ == '__main__':
    run_specified_tests([TestLogout])

F
FAIL: test_bad_logout (__main__.TestLogout)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/var/folders/tc/3ljmtg750xz6n4_dy5k9l0mm0000gn/T/ipykernel_76397/1973513340.py", line 21, in test_bad_logout
    self.assertFalse(is_number(output_responce),msg="Received Valid Temp when should have been rejected for invalid token.")
AssertionError: True is not false : Received Valid Temp when should have been rejected for invalid token.

----------------------------------------------------------------------
Ran 1 test in 0.083s

FAILED (failures=1)


### Patch Logout handling

In [87]:
# Patched Function section.
def processCommands(self, msg, addr):
    cmds = msg.split(';')
    for c in cmds:
        cs = c.split(' ')
        if len(cs) == 2:  # should be either AUTH or LOGOUT
            if cs[0] == "AUTH":
                if cs[1] == "!Q#E%T&U8i6y4r2w":
                    self.tokens.append(''.join(
                        random.choice(string.ascii_uppercase + string.ascii_lowercase + string.digits) for _ in
                        range(16)))
                    self.serverSocket.sendto(self.tokens[-1].encode("utf-8"), addr)
                else:
                    self.serverSocket.sendto(b"Bad Token\n", addr)
                    return
                    # print (self.tokens[-1])
            elif cs[0] == "LOGOUT":
                if cs[1] in self.tokens:
                    self.tokens.remove(cs[1])
                    self.serverSocket.sendto(b"Logged Out\n", addr)
                else:
                    self.serverSocket.sendto(b"Bad Token\n", addr)
                return
            else:  # unknown command
                self.serverSocket.sendto(b"Invalid Command\n", addr)
        elif c == "SET_DEGF":
            self.deg = "F"
        elif c == "SET_DEGC":
            self.deg = "C"
        elif c == "SET_DEGK":
            self.deg = "K"
        elif c == "GET_TEMP":
            self.serverSocket.sendto(b"%f\n" % self.getTemperature(), addr)
        elif c == "UPDATE_TEMP":
            self.updateTemperature()
        elif c:
            self.serverSocket.sendto(b"Invalid Command\n", addr)

## Vuln 3: Inaccurate Incubator Temperature Reporting on unit change
Both updateInfTemp and updateIncTemp use the same calculation for temperature regardless of what units the different systems are using. This has a direct impact on the integrity of the reporting of temperatures.
*SampleNetworkServer.py Lines 157 & 165*
```
self.infTemps.append(self.infTherm.getTemperature()-273)
self.incTemps.append(self.incTherm.getTemperature()-273)
```
This causes inconsistencies in the reporting of temperatures. This not only can cause unintended temperature changes in the incubator, but also causes the graph to not display the lines, as the values fall out of the visibile range, which would make it hard for a medical professional to monitor.




In [88]:

#Exploit
import unittest
import socket

class TestUnitChange(unittest.TestCase):
    # Unit change Test
    def test_unit_change(self):
        s_rl = socket.socket(family=socket.AF_INET, type=socket.SOCK_DGRAM)
        request_1=b'AUTH !asd;GET_TEMP'
        s_rl.sendto(request_1, ("127.0.0.1", 23457))
        output_responce, real_server_addr = s_rl.recvfrom(1024)
        deg_k=float(output_responce.strip())
        # print(f"deg kelvin={deg_k}")
        input_request=b'AUTH sad;SET_DEGF'
        s_rl.sendto(input_request, ("127.0.0.1", 23457))
        # output_responce, real_server_addr = s_rl.recvfrom(1024)
        s_rl.sendto(request_1, ("127.0.0.1", 23457))
        output_responce, real_server_addr = s_rl.recvfrom(1024)
        deg_f=float(output_responce.strip())
        # print(f"deg f start={deg_f}")
        request_2=b'AUTH sad;SET_DEGC'
        # request_3=b'AUTH sad;GET_TEMP'
        s_rl.sendto(input_request, ("127.0.0.1", 23457))
        s_rl.sendto(input_request, ("127.0.0.1", 23457))
        s_rl.sendto(request_1, ("127.0.0.1", 23457))
        s_rl.sendto(request_2, ("127.0.0.1", 23457))
        s_rl.sendto(request_1, ("127.0.0.1", 23457))
        output_responce, real_server_addr = s_rl.recvfrom(1024)
        s_rl.close()
        deg_f2=float(output_responce.strip())
        # print(f"deg f2={deg_f2}")
        self.assertAlmostEqual(deg_f,deg_f2, places=0,msg="Temperature shouldn't change much in time it takes to call SET_DEGF twice")
if __name__ == '__main__':
    run_specified_tests([TestUnitChange])

.
----------------------------------------------------------------------
Ran 1 test in 0.287s

OK


# Vuln 4: Lack of ability to confirm set units.
There is no public facing command to retrieve the units set despite having the commands to set the units at any time. Temperatures without a unit give little context to a medical professional. An attacker could change the units without the other party knowing using Vulns 1 and 2 and cause the medical professional to respond inappropriately as they have lost integrity of their configurations and the availability they should have gained by being able to set they dynamically. Further the command to set the units does not return any sort of confirmation or status leaving those callers often hanging awaiting a response and/or unsure of the outcome. This gaps This could lead to clients DoSing the client with the SET_DEG commands.

The remediation is to add code to the server that everytime the client requests the temperature, the units are returned as well.


## Vuln 5: Vulnerable to Decode Exceptions
The server is asssuming that the client would only send utf-8 encoded strings to the server.
If a malicious/reckless client sent say utf-16 string, it will cause a decoding exception on the server.
The server is unable to handle this, so it will crash, causing an availability issue.

The vulnerability is found within run function (line 88), where the payload from the client is decoded.

The easiest fix would be to add under line 116 the ability to catch UnicodeDecodeError exceptions. When handling this exception, the server could sent the client the message, "Bad Command" or something of that nature.
### Exploit of Decode Vuln

In [89]:
#### Exploit for Decode

import unittest
import socket
class TestDecode(unittest.TestCase):
    def test_decode_exception(self):
        s_rl = socket.socket(family=socket.AF_INET, type=socket.SOCK_DGRAM)
        request_1 = 'AUTH ϼϽϾϿ'.encode('utf-16')
        s_rl.sendto(request_1, ("127.0.0.1", 23456))
        token, real_server_addr = s_rl.recvfrom(1024)
        token = token.strip()

if __name__ == '__main__':
    run_specified_tests([TestDecode])

AttributeError: module 'unittest' has no attribute 'countTestCases'

### Patch of Decode Vuln

In [ ]:
#### Updated Run Function

    def run(self) : # fthe running function
        while True : 
            try :
                msg, addr = self.serverSocket.recvfrom(1024)
                msg = msg.decode("utf-8").strip()
                cmds = msg.split(' ')
                if len(cmds) == 1 : # protected commands case
                    semi = msg.find(';')
                    if semi != -1 : #if we found the semicolon
                        #print (msg)
                        if msg[:semi] in self.tokens : #if its a valid token
                            self.processCommands(msg[semi+1:], addr)
                        else :
                            self.serverSocket.sendto(b"Bad Token\n", addr)
                    else :
                            self.serverSocket.sendto(b"Bad Command\n", addr)
                elif len(cmds) == 2 :
                    if cmds[0] in self.open_cmds : #if its AUTH or LOGOUT
                        self.processCommands(msg, addr) 
                    else :
                        self.serverSocket.sendto(b"Authenticate First\n", addr)
                else :
                    # otherwise bad command
                    self.serverSocket.sendto(b"Bad Command\n", addr)
    
            except IOError as e :
                if e.errno == errno.EWOULDBLOCK :
                    #do nothing
                    pass
                else :
                    #do nothing for now
                    pass
                msg = ""
            # Handle Decode issues
            except UnicodeDecodeError:
                self.serverSocket.sendto(b"Bad Command, Invalid encoding\n", addr)
 

            self.updateTemperature()
            time.sleep(self.updatePeriod)


## Vuln 6: Server is unable to safely catch Keyboard Interrupts.
When the server receives a SIGINT (CTRL + C), the server will crash unexpectedly. This is a bug as in secure programming, we should not have any unhandled exceptions.

The issue is not specific to a line of code. However the quickest way to fix this is under line 116 to add the ability to catch KeyBoardIntterupt Exception. In the exception block, either break out of the while loop or use pass to ignore it. If you want to be more thorough, there could be a signal handler imported to handle SIGINT and all other signals that could be received.
### Patch of Keyboard Interrupt Handling

In [ ]:
#### Updated Run Function

    def run(self) : #the running function
        while True : 
            try :
                msg, addr = self.serverSocket.recvfrom(1024)
                msg = msg.decode("utf-8").strip()
                cmds = msg.split(' ')
                if len(cmds) == 1 : # protected commands case
                    semi = msg.find(';')
                    if semi != -1 : #if we found the semicolon
                        #print (msg)
                        if msg[:semi] in self.tokens : #if its a valid token
                            self.processCommands(msg[semi+1:], addr)
                        else :
                            self.serverSocket.sendto(b"Bad Token\n", addr)
                    else :
                            self.serverSocket.sendto(b"Bad Command\n", addr)
                elif len(cmds) == 2 :
                    if cmds[0] in self.open_cmds : #if its AUTH or LOGOUT
                        self.processCommands(msg, addr) 
                    else :
                        self.serverSocket.sendto(b"Authenticate First\n", addr)
                else :
                    # otherwise bad command
                    self.serverSocket.sendto(b"Bad Command\n", addr)
    
            except IOError as e :
                if e.errno == errno.EWOULDBLOCK :
                    #do nothing
                    pass
                else :
                    #do nothing for now
                    pass
                msg = ""
            # Handle SIGINT
            except KeyboardInterrupt:
                self.serverSocket.sendto(b"Server received SIGINT, closing\n", addr)
                break
                
            self.updateTemperature()
            time.sleep(self.updatePeriod)


In [90]:
#Consolidated Tests

if __name__ == '__main__':
    run_specified_tests([TestPassword,TestLogout,TestUnitChange])

FFF
FAIL: test_bad_password (__main__.TestPassword)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/var/folders/tc/3ljmtg750xz6n4_dy5k9l0mm0000gn/T/ipykernel_76397/2613504411.py", line 34, in test_bad_password
    self.assertFalse(is_number(output_responce),msg="Received Valid Temp when should have been rejected for invalid auth.")
AssertionError: True is not false : Received Valid Temp when should have been rejected for invalid auth.

FAIL: test_bad_logout (__main__.TestLogout)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/var/folders/tc/3ljmtg750xz6n4_dy5k9l0mm0000gn/T/ipykernel_76397/1973513340.py", line 21, in test_bad_logout
    self.assertFalse(is_number(output_responce),msg="Received Valid Temp when should have been rejected for invalid token.")
AssertionError: True is not false : Received Valid Temp when should have been rejected for invalid to